# GPT2 Text Generation with KerasHub

In this tutorial, you will learn to use [KerasHub](https://keras.io/keras_hub/) to load a
pre-trained Large Language Model (LLM) - [GPT-2 model](https://openai.com/research/better-language-models)
(originally invented by OpenAI), finetune it to a specific text style, and
generate text based on users' input (also known as prompt). You will also learn
how GPT2 adapts quickly to non-English languages, such as Chinese.

##  Before we begin

Colab offers different kinds of runtimes. Make sure to go to **Runtime ->
Change runtime type** and choose the GPU Hardware Accelerator runtime
(which should have >12G host RAM and ~15G GPU RAM) since you will finetune the
GPT-2 model. Running this tutorial on CPU runtime will take hours.

## Install KerasHub, Choose Backend and Import Dependencies

This examples uses [Keras 3](https://keras.io/keras_3/) to work in any of
`"tensorflow"`, `"jax"` or `"torch"`. Support for Keras 3 is baked into
KerasHub, simply change the `"KERAS_BACKEND"` environment variable to select
the backend of your choice. We select the JAX backend below.

In [ ]:
!pip install git+https://github.com/keras-team/keras-hub.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.18.1 requires keras-hub==0.18.1, but you have keras-hub 0.21.0.dev0 which is incompatible.


In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_hub
import keras
import tensorflow as tf
import time

keras.mixed_precision.set_global_policy("mixed_float16")

## Introduction to Generative Large Language Models (LLMs)

Large language models (LLMs) are a type of machine learning models that are
trained on a large corpus of text data to generate outputs for various natural
language processing (NLP) tasks, such as text generation, question answering,
and machine translation.

Generative LLMs are typically based on deep learning neural networks, such as
the [Transformer architecture](https://arxiv.org/abs/1706.03762) invented by
Google researchers in 2017, and are trained on massive amounts of text data,
often involving billions of words. These models, such as Google [LaMDA](https://blog.google/technology/ai/lamda/)
and [PaLM](https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html),
are trained with a large dataset from various data sources which allows them to
generate output for many tasks. The core of Generative LLMs is predicting the
next word in a sentence, often referred as **Causal LM Pretraining**. In this
way LLMs can generate coherent text based on user prompts. For a more
pedagogical discussion on language models, you can refer to the
[Stanford CS324 LLM class](https://stanford-cs324.github.io/winter2022/lectures/introduction/).

## Introduction to KerasHub

Large Language Models are complex to build and expensive to train from scratch.
Luckily there are pretrained LLMs available for use right away. [KerasHub](https://keras.io/keras_hub/)
provides a large number of pre-trained checkpoints that allow you to experiment
with SOTA models without needing to train them yourself.

KerasHub is a natural language processing library that supports users through
their entire development cycle. KerasHub offers both pretrained models and
modularized building blocks, so developers could easily reuse pretrained models
or stack their own LLM.

In a nutshell, for generative LLM, KerasHub offers:

- Pretrained models with `generate()` method, e.g.,
    `keras_hub.models.GPT2CausalLM` and `keras_hub.models.OPTCausalLM`.
- Sampler class that implements generation algorithms such as Top-K, Beam and
    contrastive search. These samplers can be used to generate text with
    custom models.

## Load a pre-trained GPT-2 model and generate some text

KerasHub provides a number of pre-trained models, such as [Google
Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
and [GPT-2](https://openai.com/research/better-language-models). You can see
the list of models available in the [KerasHub repository](https://github.com/keras-team/keras-hub/tree/master/keras_hub/src/models).

It's very easy to load the GPT-2 model as you can see below:

In [ ]:
# To speed up training and generation, we use preprocessor of length 128
# instead of full length 1024.
preprocessor = keras_hub.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_hub.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

Once the model is loaded, you can use it to generate some text right away. Run
the cells below to give it a try. It's as simple as calling a single function
*generate()*:

In [ ]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
My trip to Yosemite was a little less than three weeks ago and I have had no problems getting through it. The trail is a bit longer, so I had to get out and walk it. The hike to the summit is a little more than three hours, but I'm sure it'll be worth it. It's a good, solid, and easy way to get to the top of the mountain and get some sun and some shade. I was able to get some shade and some shade from the trail and the view. It was a good day, but it wasn't the best day of my trip so I'm not sure if it's worth it. I did enjoy a couple of the views, but I'm going to leave it at that.

I'm not the first person to go up and down the Sierra Nevada, and there are many others. The Sierra Nevada is not as beautiful or steep as some of the other places in the region. I've been there before,
TOTAL TIME ELAPSED: 11.54s


Try another one:

In [ ]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
That Italian restaurant is now a place of pilgrimage to those who've never seen one of those things before. The Italian restaurant, which was built in 2007, is a bit of a tourist attraction, with a few hundred people on hand to enjoy the food. But there's something more special about the place: the Italian restaurant's name.

The name means "the place of pilgrimage."

Italian restaurant in the heart of Milan. (Photo: Courtesy)

In the first place, you're going to have a good time. The place is open for business, with lots of activity and lots of people. It's a place of pilgrimage, and the people there are a bit more relaxed and more friendly.

The food is very good. The service is very good. The service to the restaurant is very good. And I can't say that I'm surprised by it. It's a place of pilgrimage.

But I can say that it is also a tourist
TOTAL TIME ELAPSED: 1.12s


Notice how much faster the second call is. This is because the computational
graph is [XLA compiled](https://www.tensorflow.org/xla) in the 1st run and
re-used in the 2nd behind the scenes.

The quality of the generated text looks OK, but we can improve it via
fine-tuning.

## More on the GPT-2 model from KerasHub

Next up, we will actually fine-tune the model to update its parameters, but
before we do, let's take a look at the full set of tools we have to for working
with for GPT2.

The code of GPT2 can be found
[here](https://github.com/keras-team/keras-hub/tree/master/keras_hub/src/models/gpt2).
Conceptually the `GPT2CausalLM` can be hierarchically broken down into several
modules in KerasHub, all of which have a *from_preset()* function that loads a
pretrained model:

- `keras_hub.models.GPT2Tokenizer`: The tokenizer used by GPT2 model, which is a
    [byte-pair encoder](https://huggingface.co/course/chapter6/5?fw=pt).
- `keras_hub.models.GPT2CausalLMPreprocessor`: the preprocessor used by GPT2
    causal LM training. It does the tokenization along with other preprocessing
    works such as creating the label and appending the end token.
- `keras_hub.models.GPT2Backbone`: the GPT2 model, which is a stack of
    `keras_hub.layers.TransformerDecoder`. This is usually just referred as
    `GPT2`.
- `keras_hub.models.GPT2CausalLM`: wraps `GPT2Backbone`, it multiplies the
    output of `GPT2Backbone` by embedding matrix to generate logits over
    vocab tokens.

## Finetune on Reddit dataset

Now you have the knowledge of the GPT-2 model from KerasHub, you can take one
step further to finetune the model so that it generates text in a specific
style, short or long, strict or casual. In this tutorial, we will use reddit
dataset for example.

In [ ]:
import tensorflow_datasets as tfds

reddit_ds = tfds.load("reddit_tifu", split="train", as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/reddit_tifu/short/incomplete.ALXF6E_1.1.2/reddit_tifu-train.tfrecord*...: …

Dataset reddit_tifu downloaded and prepared to /root/tensorflow_datasets/reddit_tifu/short/1.1.2. Subsequent calls will reuse this data.


Let's take a look inside sample data from the reddit TensorFlow Dataset. There
are two features:

- **__document__**: text of the post.
- **__title__**: the title.

In [ ]:
for document, title in reddit_ds:
    print(document.numpy())
    print(title.numpy())
    break

b"me and a friend decided to go to the beach last sunday. we loaded up and headed out. we were about half way there when i decided that i was not leaving till i had seafood. \n\nnow i'm not talking about red lobster. no friends i'm talking about a low country boil. i found the restaurant and got directions. i don't know if any of you have heard about the crab shack on tybee island but let me tell you it's worth it. \n\nwe arrived and was seated quickly. we decided to get a seafood sampler for two and split it. the waitress bought it out on separate platters for us. the amount of food was staggering. two types of crab, shrimp, mussels, crawfish, andouille sausage, red potatoes, and corn on the cob. i managed to finish it and some of my friends crawfish and mussels. it was a day to be a fat ass. we finished paid for our food and headed to the beach. \n\nfunny thing about seafood. it runs through me faster than a kenyan \n\nwe arrived and walked around a bit. it was about 45min since we a

In our case, we are performing next word prediction in a language model, so we
only need the 'document' feature.

In [ ]:
train_ds = (
    reddit_ds.map(lambda document, _: document)
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

Now you can finetune the model using the familiar *fit()* function. Note that
`preprocessor` will be automatically called inside `fit` method since
`GPT2CausalLM` is a `keras_hub.models.Task` instance.

This step takes quite a bit of GPU memory and a long time if we were to train
it all the way to a fully trained state. Here we just use part of the dataset
for demo purposes.

In [ ]:
train_ds = train_ds.take(500)
num_epochs = 1

# Linearly decaying learning rate.
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 323ms/step - accuracy: 0.3123 - loss: 3.3946


After fine-tuning is finished, you can again generate text using the same
*generate()* function. This time, the text will be closer to Reddit writing
style, and the generated length will be close to our preset length in the
training set.

In [ ]:
start = time.time()

output = gpt2_lm.generate("The Italian restaurant is", max_length=200)  # I like basketball
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
The Italian restaurant is a great way to get away from a busy busy day.

i'm on vacation and have a couple of days to myself before i leave. i have a small room with a large bed, but it's not too far away from the restaurant. 

so, i'm sitting on the couch with my laptop, and i start to get bored. i'm not a huge fan of loud music and i'm not sure how long i can stay on it until i finish my meal. so, i go to bed, and i wake up the next morning, and i don’t hear anything.
TOTAL TIME ELAPSED: 1.89s


## Into the Sampling Method

In KerasHub, we offer a few sampling methods, e.g., contrastive search,
Top-K and beam sampling. By default, our `GPT2CausalLM` uses Top-k search, but
you can choose your own sampling method.

Much like optimizer and activations, there are two ways to specify your custom
sampler:

- Use a string identifier, such as "greedy", you are using the default
configuration via this way.
- Pass a `keras_hub.samplers.Sampler` instance, you can use custom configuration
via this way.

In [ ]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_hub.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball, but not as much as basketball. 

so, i'm in high school and my mom's a huge basketball fan. she's also the best basketball player she has ever had. 

she's a pretty good basketball player and she likes to play with my dad. so we go to the local game, and we're talking about the game and the game starts and i'm just sitting on the court and i look at the ball in my hand, and she's like, "what the fuck is going on?" and i'm like, "i'm not sure. i'm not sure if the ball is going to go to my dad. 

GPT-2 output:
I like basketball, but i don't like to play it. 

so i was playing basketball at my local high school, and my friend and i were playing a game of basketball. 

we were playing a game of basketball, and i was playing with my friend, and he was playing with his friend's friend. 

so i was playing with my friend's friend, and he was playing with his friend's friend's friend's friend's friend's friend's friend's friend's friend's friend's friend's frie

For more details on KerasHub `Sampler` class, you can check the code
[here](https://github.com/keras-team/keras-hub/tree/master/keras_hub/src/samplers).

## Finetune on Chinese Poem Dataset

We can also finetune GPT2 on non-English datasets. For readers knowing Chinese,
this part illustrates how to fine-tune GPT2 on Chinese poem dataset to teach our
model to become a poet!

Because GPT2 uses byte-pair encoder, and the original pretraining dataset
contains some Chinese characters, we can use the original vocab to finetune on
Chinese dataset.

In [ ]:
!# Load chinese poetry dataset.
!git clone https://github.com/chinese-poetry/chinese-poetry.git

Cloning into 'chinese-poetry'...
remote: Enumerating objects: 7341, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 7341 (delta 7), reused 3 (delta 3), pack-reused 7330 (from 2)
Receiving objects: 100% (7341/7341), 236.99 MiB | 16.64 MiB/s, done.
Resolving deltas: 100% (5012/5012), done.
Updating files: 100% (2285/2285), done.


Load text from the json file. We only use《全唐诗》for demo purposes.

In [ ]:
import os
import json

poem_collection = []
for file in os.listdir("chinese-poetry/全唐诗"):
    if ".json" not in file or "poet" not in file:
        continue
    full_filename = "%s/%s" % ("chinese-poetry/全唐诗", file)
    with open(full_filename, "r") as f:
        content = json.load(f)
        poem_collection.extend(content)

paragraphs = ["".join(data["paragraphs"]) for data in poem_collection]

Let's take a look at sample data.

In [ ]:
print(paragraphs[0]), len(paragraphs)

一飽欣然下箸餘，庾郎鮭菜復何須。當時獻納堪憐許，只解區區說䱥魚。


(None, 311855)

In [ ]:
import gdown
gdown.download(url="https://drive.google.com/file/d/1-065EvxQepsQWTpDbdAfgPw0kkiwfMcR/view?usp=sharing", fuzzy=True, output='tamil.txt')

with open('tamil.txt') as ftam:
  tamil_texts = ftam.readlines()
  ftam.close()



அதே போலீஸ் தலைவர்தான் போலீசார் நல்லவர்கள் என்று தொடர்ந்து கூறி வருகிறார் .
 50000


In [ ]:
print(tamil_texts[1000], len(tamil_texts))

In [ ]:
paragraphs = tamil_texts

Similar as Reddit example, we convert to TF dataset, and only use partial data
to train.

In [ ]:
train_ds = (
    tf.data.Dataset.from_tensor_slices(paragraphs)
    .batch(16)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Running through the whole dataset takes long, only take `500` and run 1
# epochs for demo purposes.
train_ds = train_ds.take(500)
num_epochs = 1

learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-4,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

500/500 ━━━━━━━━━━━━━━━━━━━━ 134s 192ms/step - accuracy: 0.6215 - loss: 1.4723


Let's check the result!

In [ ]:
output = gpt2_lm.generate("அதே தலைவர்தான்", max_length=200)  #"昨夜雨疏风骤"
print(output)

அதே தலைவர்தான் அரசிக்கிறியத்தினவன் சியான வொல


In [ ]:
output = gpt2_lm.generate("I like basketball", max_length=200)
print(output)

I like basketballந்த பட்டத்து நடிய படத்திலான அரசி அவெர்கள் .



Not bad 😀